<a href="https://colab.research.google.com/github/Johoodcoder/CS490Project/blob/hood/Notebooks/CS490Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Non-preinstalled module installs

In [1]:
!pip install pytorch-pretrained-bert
!pip install pytorch-nlp

Import Dataset used in https://towardsdatascience.com/fake-news-classification-with-bert-afbeee601f41

In [2]:
# from google.colab import files

# uploaded = files.upload()
# fileName = ''

# for fn in uploaded.keys():
#   print('User uploaded file "{name}" with length {length} bytes'.format(
#       name=fn, length=len(uploaded[fn])))
#   fileName = fn

The base code from https://github.com/spierre91/medium_code/blob/master/fake_news_classifcation.py

In [3]:
import pandas as pd 
import numpy as np 
import torch.nn as nn
from pytorch_pretrained_bert import BertTokenizer, BertModel
import torch
from torchnlp.datasets import imdb_dataset
from keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import classification_report

In [4]:
pd.set_option('display.max_columns', None)
train_data, test_data = imdb_dataset(train=True, test=True)
df = pd.read_csv("condensed_fake_real_news.csv")
df = df[['text', 'type']]
print(len(df))

8000


In [5]:
from collections import Counter 

print(Counter(df['type'].values))

Counter({'fake': 4000, 'real': 4000})


In [6]:
df = df[df['type'].isin(['fake', 'real'])]
df.dropna(inplace = True)
df = df.sample(frac=1, random_state = 24).reset_index(drop=True)

print(Counter(df['type'].values))

Counter({'fake': 4000, 'real': 4000})


In [7]:
train_data_df = df.head(6400)
test_data_df = df.tail(1600)
print(train_data_df)

                                                   text  type
0     Donald Trump s most recent secretive actions t...  fake
1     WASHINGTON (Reuters) - U.S. President Donald T...  real
2     WASHINGTON (Reuters) - U.S. President Donald T...  real
3     WASHINGTON (Reuters) - Congressional leaders a...  real
4     One of Trump s biggest campaign promises was t...  fake
...                                                 ...   ...
6395  The inauguration of President-elect Donald Tru...  fake
6396  WASHINGTON (Reuters) - The U.S. Congress’ watc...  real
6397  Former First Lady Hillary Clinton doesn t twee...  fake
6398  WASHINGTON (Reuters) - The solidly Republican ...  real
6399  The GOP has long been a place that only held t...  fake

[6400 rows x 2 columns]


In [8]:
train_data = []
for index, row in train_data_df.iterrows():
    train_data.append({'text': row['text'], 'type': row['type']})

test_data = []
for index, row in test_data_df.iterrows():
    test_data.append({'text': row['text'], 'type': row['type']})

# train_data = [{'text': text, 'type': type_data } for text in list(train_data['text']) for type_data in list(train_data['type'])]
# test_data = [{'text': text, 'type': type_data } for text in list(test_data['text']) for type_data in list(test_data['type'])]
# #gc.collect()

In [9]:
train_texts, train_labels = list(zip(*map(lambda d: (d['text'], d['type']), train_data)))
test_texts, test_labels = list(zip(*map(lambda d: (d['text'], d['type']), test_data)))

In [10]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
train_tokens = list(map(lambda t: ['[CLS]'] + tokenizer.tokenize(t)[:511], train_texts))
test_tokens = list(map(lambda t: ['[CLS]'] + tokenizer.tokenize(t)[:511], test_texts))

train_tokens_ids = list(map(tokenizer.convert_tokens_to_ids, train_tokens))
test_tokens_ids = list(map(tokenizer.convert_tokens_to_ids, test_tokens))



train_tokens_ids = pad_sequences(train_tokens_ids, maxlen=512, truncating="post", padding="post", dtype="int")
test_tokens_ids = pad_sequences(test_tokens_ids, maxlen=512, truncating="post", padding="post", dtype="int")


In [11]:
train_y = np.array(train_labels) == 'fake'
test_y = np.array(test_labels) == 'fake'

In [12]:
class BertBinaryClassifier(nn.Module):
    def __init__(self, dropout=0.1):
        super(BertBinaryClassifier, self).__init__()

        self.bert = BertModel.from_pretrained('bert-base-uncased')

        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(768, 1)
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, tokens, masks=None):
        _, pooled_output = self.bert(tokens, attention_mask=masks, output_all_encoded_layers=False)
        dropout_output = self.dropout(pooled_output)
        linear_output = self.linear(dropout_output)
        proba = self.sigmoid(linear_output)
        return proba

In [13]:
train_masks = [[float(i > 0) for i in ii] for ii in train_tokens_ids]
test_masks = [[float(i > 0) for i in ii] for ii in test_tokens_ids]
train_masks_tensor = torch.tensor(train_masks)
test_masks_tensor = torch.tensor(test_masks)

In [14]:
train_tokens_tensor = torch.tensor(train_tokens_ids)
train_y_tensor = torch.tensor(train_y.reshape(-1, 1)).float()
test_tokens_tensor = torch.tensor(test_tokens_ids)
test_y_tensor = torch.tensor(test_y.reshape(-1, 1)).float()

In [15]:
BATCH_SIZE = 1
EPOCHS = 1

train_dataset =  torch.utils.data.TensorDataset(train_tokens_tensor, train_masks_tensor, train_y_tensor)
train_sampler =  torch.utils.data.RandomSampler(train_dataset)
train_dataloader =  torch.utils.data.DataLoader(train_dataset, sampler=train_sampler, batch_size=BATCH_SIZE)
test_dataset =  torch.utils.data.TensorDataset(test_tokens_tensor, test_masks_tensor, test_y_tensor)
test_sampler =  torch.utils.data.SequentialSampler(test_dataset)
test_dataloader =  torch.utils.data.DataLoader(test_dataset, sampler=test_sampler, batch_size=BATCH_SIZE)

In [ ]:
bert_clf = BertBinaryClassifier()
optimizer = torch.optim.Adam(bert_clf.parameters(), lr=3e-6)

for epoch_num in range(EPOCHS):
    bert_clf.train()
    train_loss = 0
    for step_num, batch_data in enumerate(train_dataloader):
        token_ids, masks, labels = tuple(t for t in batch_data)
        probas = bert_clf(token_ids, masks)
        loss_func = nn.BCELoss()
        batch_loss = loss_func(probas, labels)
        train_loss += batch_loss.item()
        bert_clf.zero_grad()
        batch_loss.backward()
        optimizer.step()
        print('Epoch: ', epoch_num + 1)
        print("\r" + "{0}/{1} loss: {2} ".format(step_num, len(train_data) / BATCH_SIZE, train_loss / (step_num + 1)))

Streaming output truncated to the last 5000 lines.
Epoch:  1
1671/6400.0 loss: 0.12454708651990438 
Epoch:  1
1672/6400.0 loss: 0.12447733248378558 
Epoch:  1
1673/6400.0 loss: 0.1244073857733003 
Epoch:  1
1674/6400.0 loss: 0.1243384507440253 
Epoch:  1
1675/6400.0 loss: 0.12426888103305689 
Epoch:  1
1676/6400.0 loss: 0.12419961429366005 
Epoch:  1
1677/6400.0 loss: 0.12413056581464282 
Epoch:  1
1678/6400.0 loss: 0.1240616939183127 
Epoch:  1
1679/6400.0 loss: 0.12399343847175173 
Epoch:  1
1680/6400.0 loss: 0.1239238591786587 
Epoch:  1
1681/6400.0 loss: 0.12385424878946577 
Epoch:  1
1682/6400.0 loss: 0.12378633022529698 
Epoch:  1
1683/6400.0 loss: 0.1237172265390527 
Epoch:  1
1684/6400.0 loss: 0.12365131485151563 
Epoch:  1
1685/6400.0 loss: 0.12358299536218431 
Epoch:  1
1686/6400.0 loss: 0.12351449470935034 
Epoch:  1
1687/6400.0 loss: 0.12344599353294088 
Epoch:  1
1688/6400.0 loss: 0.12337717618942129 
Epoch:  1
1689/6400.0 loss: 0.12330930049409924 
Epoch:  1
1690/6400.0 l

In [ ]:
bert_clf.eval()
bert_predicted = []
all_logits = []
with torch.no_grad():
    for step_num, batch_data in enumerate(test_dataloader):

        token_ids, masks, labels = tuple(t for t in batch_data)

        logits = bert_clf(token_ids, masks)
        loss_func = nn.BCELoss()
        loss = loss_func(logits, labels)
        numpy_logits = logits.cpu().detach().numpy()
        
        bert_predicted += list(numpy_logits[:, 0] > 0.5)
        all_logits += list(numpy_logits[:, 0])
        
print(classification_report(test_y, bert_predicted))
